## PDI - Phuture DeFi Index (Multi-chain) - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  0.5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["decentralized-finance-defi"]

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                'arbitrum-one',
                                min_mcap,
                                min_weight,
                                max_weight,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                100000,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    remove_category_assets=['the-graph','staked-ether','injective-protocol','chainlink','seedify-fund']
    )
liquidity

Excluding chainflip, pricing data available only for 149 < 180 days
Excluding stakewise-v3-oseth, pricing data available only for 131 < 180 days
Excluding vertex-protocol, pricing data available only for 152 < 180 days
xdai not supported
Manually calculating slippage for uniswap on avalanche
near-protocol not supported
energi not supported
huobi-token not supported
harmony-shard-0 not supported
sora not supported
Manually calculating slippage for maker on avalanche
energi not supported
sora not supported
sora not supported
near-protocol not supported
huobi-token not supported
harmony-shard-0 not supported
energi not supported
harmony-shard-0 not supported
havven on avalanche returned an invalid API response
near-protocol not supported
huobi-token not supported
energi not supported
energi not supported
xdai not supported
Manually calculating slippage for pancakeswap-token on ethereum
aptos not supported
Manually calculating slippage for frax-ether on ethereum
moonbeam not supported
frax

,arbitrum-one,avalanche,base,binance-smart-chain,ethereum,fantom,optimistic-ethereum,polygon-pos,optimal chain,best slippage,best slippage chain
id,,,,,,,,,,,
staked-frax-ether,-0.002836,NaN,NaN,NaN,-0.007546,NaN,NaN,NaN,arbitrum-one,-0.002836,arbitrum-one
tbtc,-0.003391,NaN,-0.058787,NaN,-0.004345,NaN,-0.421946,-0.065331,arbitrum-one,-0.003391,arbitrum-one
maker,NaN,-0.999190,NaN,NaN,-0.006261,NaN,NaN,-0.843329,ethereum,-0.006261,ethereum
uniswap,-0.008062,-0.996934,NaN,-0.192071,-0.006539,NaN,-0.940009,-0.274628,arbitrum-one,-0.006539,ethereum
pancakeswap-token,NaN,NaN,NaN,-0.007228,-0.380415,NaN,NaN,NaN,binance-smart-chain,-0.007228,binance-smart-chain
...,...,...,...,...,...,...,...,...,...,...,...
tellor,NaN,NaN,NaN,NaN,-0.933304,NaN,NaN,NaN,None,-0.933304,ethereum
hashflow,NaN,NaN,NaN,-0.950881,-0.941259,NaN,NaN,NaN,None,-0.941259,ethereum
stp-network,NaN,NaN,NaN,NaN,-0.953475,NaN,NaN,NaN,None,-0.953475,ethereum


## Show Results

In [4]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain
id,,,,,,
uniswap,UNI,5664592447,0.302583,19830,0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0,arbitrum-one
maker,MKR,2728582845,0.151634,9938,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,ethereum
lido-dao,LDO,1764062741,0.098033,6425,0x5a98fcbea516cf06857215779fd812ca3bef1b32,ethereum
aave,AAVE,1265231102,0.070312,4608,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,ethereum
ribbon-finance,RBN,972923974,0.054068,3544,0x6123b0049f904d730db3c36a31167d9d4121fa6b,ethereum
havven,SNX,962351989,0.053480,3505,0x8700daec35af8ff88c16bdf0418774cb3d7599b4,optimistic-ethereum
gnosis,GNO,890665597,0.049496,3244,0x6810e776880c02933d47db1b9fc05908e5386b96,ethereum
pancakeswap-token,CAKE,690511871,0.038373,2515,0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82,binance-smart-chain
pendle,PENDLE,553024884,0.030733,2015,0x0c880f6761f1af8d9aa9c466984b80dab9a8c9e8,arbitrum-one


In [5]:
from backtester import BackTester, convert_results_table

bt = BackTester(convert_results_table(results),['defipulse-index'])

data, chart = bt.assess_different_rebalancing_periods([30,60,90],0.02)

chart

/Users/olivermehr/Documents/GitHub/rebalancing-data/PDI/../backtester.py:55: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  self.raw_data = self.raw_data.pct_change()


defipulse-index


/Users/olivermehr/.local/share/virtualenvs/rebalancing-data-VGaZtODH/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [6]:
data

,Index cum-return w/ 30 days rebalancing window,Index cum-return w/ 60 days rebalancing window,Index cum-return w/ 90 days rebalancing window,defipulse-index-cum-rtn
2022-03-18,0.000000,0.000000,0.000000,NaN
2022-03-19,0.026211,0.026211,0.026211,0.000000
2022-03-20,0.047490,0.047490,0.047490,0.030899
2022-03-21,0.000433,0.000433,0.000433,-0.018022
2022-03-22,0.032738,0.032738,0.032738,-0.013608
...,...,...,...,...
2024-04-15,-0.176384,0.060743,0.143753,-0.401867
2024-04-16,-0.199019,0.035757,0.116811,-0.422859
2024-04-17,-0.196753,0.043854,0.125542,-0.407309
2024-04-18,-0.224228,0.010604,0.089690,-0.430829
